In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import ast
pd.set_option('display.max_columns', None)

In [2]:
# Import cleaned data
buys = pd.read_csv('clean_buys.csv', index_col=0)
rents = pd.read_csv('clean_rents.csv', index_col=0)

In [3]:
buys.head()

,url,street_address,city,state,zipcode,neighborhood,beds,baths,square_footage,price,listing_type,description,details,home_type,lot_size,year_built,num_rooms,heating,heating_fuel,air_con,microwave,dishwasher,disposal,washer,dryer,fireplace,vaulted_ceiling,double_pane,garage,num_parking,security,num_stories,foundation_type,patio,porch,deck,pool,rooms_Dining Room,rooms_Family Room,rooms_Laundry Room,rooms_Walk In Closet,rooms_Pantry,rooms_Breakfast Nook,rooms_Office,rooms_Workshop,rooms_Library,rooms_Recreation Room,rooms_Master Bedroom,rooms_Living Room,rooms_Loft,cool_Central,cool_Evaporative,cool_Wall,cool_None,cool_Solar,cool_Refrigeration,floors_Carpet,floors_Hardwood,floors_Laminate,floors_Tile,floors_Linoleum Vinyl,floors_Concrete,floors_Slate,parking_Detached Garage,parking_Attached Garage,parking_Carport,parking_Off Street,parking_On Street,parking_None,parking_Garage Attached,exteriors_Brick,exteriors_Wood,exteriors_Stucco,exteriors_Stone,exteriors_Cement,exteriors_Concrete,exteriors_Vinyl,exteriors_Metal,exteriors_Shingle,exteriors_Composition,exteriors_Products
0,https://www.trulia.com/p/ca/sacramento/3141-ye...,3141 Yellowstone Ln,Sacramento,CA,95821,NaN,4.0,2.0,1689.0,390000,BUY,"Beautifully upgrade, large 4 bedroom, 2 full b...","['Single Family Home', '$231/sqft', 'Lot Size:...",Single Family Home,11325.6,1946.0,8.0,Forced Air,Electric,Air Conditioning,NaN,NaN,NaN,Washer,Dryer,Fireplace,NaN,Double Paned Windows,Garage,1.0,Security System,1.0,NaN,NaN,NaN,NaN,NaN,Dining Room,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Central,NaN,NaN,NaN,NaN,NaN,Carpet,Hardwood,NaN,NaN,NaN,NaN,NaN,Detached Garage,NaN,NaN,NaN,NaN,NaN,NaN,Brick,Wood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.trulia.com/p/ca/sacramento/7748-fi...,7748 Finnhorse Way,Sacramento,CA,95828,NaN,5.0,3.0,3020.0,529900,BUY,"Fabulous 5 bedroom, 3 bathroom, two-story prop...","['Single Family Home', '$175/sqft', 'Lot Size:...",Single Family Home,6199.0,2016.0,9.0,Forced Air,Gas,Air Conditioning,NaN,Dishwasher,NaN,NaN,NaN,Fireplace,NaN,Double Paned Windows,Garage,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,Dining Room,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Central,NaN,NaN,NaN,NaN,NaN,Carpet,NaN,Laminate,Tile,NaN,NaN,NaN,NaN,Attached Garage,NaN,NaN,NaN,NaN,NaN,NaN,Wood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.trulia.com/p/ca/rancho-cordova/224...,2246 Palmwood Ct,Rancho Cordova,CA,95670,Mills Ranch,4.0,2.0,1420.0,368900,BUY,"Beautifully updated 4 bedroom, 2 bath single s...","['Single Family Home', '$260/sqft', 'Lot Size:...",Single Family Home,4729.0,1983.0,7.0,Other,NaN,Air Conditioning,Microwave,Dishwasher,Disposal,NaN,NaN,Fireplace,Vaulted Ceiling,Double Paned Windows,Garage,2.0,Security System,1.0,Slab,NaN,NaN,NaN,NaN,NaN,Family Room,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tile,NaN,NaN,NaN,NaN,Attached Garage,NaN,NaN,NaN,NaN,NaN,NaN,Wood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.trulia.com/p/ca/sacramento/5741-re...,5741 Rexleigh Dr,Sacramento,CA,95823,Valley High-North Laguna,4.0,3.0,2918.0,470000,BUY,"Beautiful 4 bedroom, 3 bathroom, 2 story locat...","['Single Family Home', '$161/sqft', 'Lot Size:...",Single Family Home,7439.0,1998.0,9.0,Forced Air,Gas,Air Conditioning,Microwave,Dishwasher,Disposal,NaN,NaN,Fireplace,NaN,Double Paned Windows,Garage,2.0,NaN,2.0,Slab,NaN,NaN,NaN,NaN,Dining Room,Family Room,Laundry Room,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Central,NaN,NaN,NaN,NaN,NaN,Carpet,NaN,Laminate,Tile,NaN,NaN,NaN,NaN,Attached Garage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.trulia.com/p/ca/sacramento/6101-go...,6101 Golden Dawn Way,Sacramento,CA,95841,Foothill Farms,4.0,3.0,1903.0,375000,BUY,This home has an attached in-laws unit with it...,"['Single Family Home', '$197/sqft', 'Lot Size:...",Single Family Home,6970.0,1955.0,7.0,Forced Air,Electric,Air Conditioning,NaN,Dishwasher,NaN,NaN,NaN,Fireplace,NaN,Double Paned Windows,Garage,2.0,NaN,1.0,NaN,

In [4]:
rents.head()

,url,street_address,city,state,zipcode,neighborhood,beds,baths,square_footage,price,listing_type,description,details,home_type,year_built,deposit,smoking,living_room,dishwasher,microwave,refrigerator,on_site_maint,on_site_mng,laundry,air_con,disposal,dryer,patio,pool,balcony,washer,basketball,ceiling_fan,fireplace,fitness,playground,floor_types,bbq,vaulted_ceiling,rparking_On Street,rparking_Off Street,rparking_Garage Detached,rparking_Garage Attached,pets_No pets allowed,pets_Cats allowed,pets_Small dogs allowed,pets_large dogs allowed,util_Sewage,util_Garbage,util_Water,util_Hot Water,util_Internet,util_Cable,util_Electricity,util_Gas,util_Heat,util_Phone
2318,https://www.trulia.com/p/ca/fair-oaks/4830-kit...,4830 Kitty Hawk St,Fair Oaks,CA,95628,NaN,3.0,2.0,1043.0,1650,RENT,"Yes, we are accepting applications NOW. All li...","['Townhouse', 'No pets allowed', 'Built in 195...",Townhouse,1958.0,NaN,No smoking,Living room,Dishwasher,Microwave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Balcony,NaN,NaN,NaN,Fireplace,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No pets allowed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2320,https://www.trulia.com/p/ca/sacramento/2604-h-...,2604 H St,Sacramento,CA,95816,Marshall School,2.0,2.0,1000.0,1400,RENT,"Yes, we are accepting applications NOW. All li...","['Multi Family', 'No pets allowed', 'Built in ...",Multi Family,1950.0,NaN,No smoking,NaN,NaN,NaN,Refrigerator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Balcony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No pets allowed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2321,https://www.trulia.com/p/ca/sacramento/2418-t-...,2418 T St #6,Sacramento,CA,95816,Newton Booth,2.0,1.5,750.0,1695,RENT,Waived $49/per adult app fee for a limited tim...,"['Multi Family', 'Cats, small dogs allowed', '...",Multi Family,1972.0,NaN,NaN,NaN,NaN,NaN,Refrigerator,NaN,NaN,Shared,Air Conditioning,NaN,NaN,NaN,NaN,Balcony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cats allowed,Small dogs allowed,NaN,Sewage,Garbage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2322,https://www.trulia.com/p/ca/sacramento/2405-f-...,2405 F St #4,Sacramento,CA,95816,Marshall School,2.0,2.0,850.0,1625,RENT,MIDTOWN / RICHMOND GROVE Spacious Upstairs Two...,"['Multi Family', 'Available: Apr 30th', 'Cats,...",Multi Family,1972.0,NaN,NaN,NaN,Dishwasher,Microwave,Refrigerator,NaN,NaN,Shared,Air Conditioning,Disposal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cats allowed,Small dogs allowed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2323,https://www.trulia.com/p/ca/sacramento/1328-d-...,1328 D St #9,Sacramento,CA,95814,Mansion Flats,2.0,2.0,1100.0,1625,RENT,Waived $49/per adult app fee for a limited tim...,"['Multi Family', 'Cats, small dogs allowed', '...",Multi Family,1972.0,NaN,NaN,NaN,Dishwasher,Microwave,Refrigerator,NaN,NaN,Shared,Air Conditioning,Disposal,NaN,NaN,NaN,Balcony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cats allowed,Small dogs allowed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
buys.columns

Index(['url', 'street_address', 'city', 'state', 'zipcode', 'neighborhood',
       'beds', 'baths', 'square_footage', 'price', 'listing_type',
       'description', 'details', 'home_type', 'lot_size', 'year_built',
       'num_rooms', 'heating', 'heating_fuel', 'air_con', 'microwave',
       'dishwasher', 'disposal', 'washer', 'dryer', 'fireplace',
       'vaulted_ceiling', 'double_pane', 'garage', 'num_parking', 'security',
       'num_stories', 'foundation_type', 'patio', 'porch', 'deck', 'pool',
       'rooms_Dining Room', 'rooms_Family Room', 'rooms_Laundry Room',
       'rooms_Walk In Closet', 'rooms_Pantry', 'rooms_Breakfast Nook',
       'rooms_Office', 'rooms_Workshop', 'rooms_Library',
       'rooms_Recreation Room', 'rooms_Master Bedroom', 'rooms_Living Room',
       'rooms_Loft', 'cool_Central', 'cool_Evaporative', 'cool_Wall',
       'cool_None', 'cool_Solar', 'cool_Refrigeration', 'floors_Carpet',
       'floors_Hardwood', 'floors_Laminate', 'floors_Tile',
       'floors_L

In [6]:
# Separate data by numeric and non-numeric
# Non-numeric needs to be encoded
# Zipcode is separate because get_dummies interprets it by default as numeric
X_buys_le = buys[['city','neighborhood','home_type', 'heating', 'heating_fuel', 'air_con', 'microwave','dishwasher', 
    'disposal', 'washer', 'dryer', 'fireplace','vaulted_ceiling', 'double_pane', 'garage', 'security', 
    'foundation_type', 'patio', 'porch', 'deck', 'pool','rooms_Dining Room', 'rooms_Family Room','rooms_Laundry Room',
    'rooms_Walk In Closet', 'rooms_Pantry', 'rooms_Breakfast Nook','rooms_Office', 'rooms_Workshop', 'rooms_Library',
    'rooms_Recreation Room','rooms_Master Bedroom','rooms_Living Room','rooms_Loft','cool_Central','cool_Evaporative',
    'cool_Wall','cool_None', 'cool_Solar', 'cool_Refrigeration', 'floors_Carpet','floors_Hardwood', 'floors_Laminate',
    'floors_Tile','floors_Linoleum Vinyl', 'floors_Concrete', 'floors_Slate','parking_Detached Garage', 
    'parking_Attached Garage', 'parking_Carport','parking_Off Street', 'parking_On Street', 'parking_None',
    'parking_Garage Attached', 'exteriors_Brick', 'exteriors_Wood','exteriors_Stucco', 'exteriors_Stone', 
    'exteriors_Cement','exteriors_Concrete', 'exteriors_Vinyl', 'exteriors_Metal','exteriors_Shingle', 
    'exteriors_Composition', 'exteriors_Products']]
X_buys_num = buys[['beds','baths','square_footage','lot_size','year_built','num_rooms','num_parking','num_stories']]
X_buys_zip = buys[['zipcode']]
y_buys = buys[['price']]

In [7]:
# One Hot Encode non-numeric data with pd.get_dummies
X_buys_zip = pd.get_dummies(X_buys_zip.astype(str))
X_buys_le = pd.get_dummies(X_buys_le)
X_buys_le.shape

(2151, 234)

In [8]:
# Rejoin data into a single dataframe
buys_encoded = pd.concat([X_buys_le,X_buys_zip, X_buys_num, y_buys], axis=1)
buys_encoded.shape

(2151, 291)

In [9]:
rents.columns

Index(['url', 'street_address', 'city', 'state', 'zipcode', 'neighborhood',
       'beds', 'baths', 'square_footage', 'price', 'listing_type',
       'description', 'details', 'home_type', 'year_built', 'deposit',
       'smoking', 'living_room', 'dishwasher', 'microwave', 'refrigerator',
       'on_site_maint', 'on_site_mng', 'laundry', 'air_con', 'disposal',
       'dryer', 'patio', 'pool', 'balcony', 'washer', 'basketball',
       'ceiling_fan', 'fireplace', 'fitness', 'playground', 'floor_types',
       'bbq', 'vaulted_ceiling', 'rparking_On Street', 'rparking_Off Street',
       'rparking_Garage Detached', 'rparking_Garage Attached',
       'pets_No pets allowed', 'pets_Cats allowed', 'pets_Small dogs allowed',
       'pets_large dogs allowed', 'util_Sewage', 'util_Garbage', 'util_Water',
       'util_Hot Water', 'util_Internet', 'util_Cable', 'util_Electricity',
       'util_Gas', 'util_Heat', 'util_Phone'],
      dtype='object')

In [10]:
# Repeat encoding process for rents
X_rents_le = rents[['city','neighborhood','home_type','smoking', 'living_room', 'dishwasher', 'microwave', 
    'refrigerator','on_site_maint', 'on_site_mng', 'laundry', 'air_con', 'disposal','dryer', 'patio', 'pool',
    'balcony', 'washer', 'basketball','ceiling_fan', 'fireplace', 'fitness', 'playground', 'floor_types','bbq', 
    'vaulted_ceiling', 'rparking_On Street', 'rparking_Off Street','rparking_Garage Detached', 
    'rparking_Garage Attached','pets_No pets allowed', 'pets_Cats allowed', 'pets_Small dogs allowed',
    'pets_large dogs allowed', 'util_Sewage', 'util_Garbage', 'util_Water','util_Hot Water', 'util_Internet', 
    'util_Cable', 'util_Electricity','util_Gas', 'util_Heat', 'util_Phone']]
X_rents_num = rents[['beds','baths','square_footage','year_built']]
X_rents_zip = rents[['zipcode']]
y_rents = rents[['price']]

In [11]:
X_rents_zip = pd.get_dummies(X_rents_zip.astype(str))
X_rents_le = pd.get_dummies(X_rents_le)
X_rents_le.shape

(1007, 181)

In [12]:
rents_encoded = pd.concat([X_rents_le,X_rents_zip, X_rents_num, y_rents], axis=1)
rents_encoded.shape

(1007, 228)

In [13]:
# Export encoded data
buys_encoded.to_csv('buys_encoded.csv')
rents_encoded.to_csv('rents_encoded.csv')
print('Export Complete')

Export Complete


In [14]:
rents_encoded.head()

,city_Antelope,city_Carmichael,city_Citrus Heights,city_Elk Grove,city_Fair Oaks,city_Folsom,city_Gold River,city_North Highlands,city_Orangevale,city_Rancho Cordova,city_Rio Linda,city_Sacramento,city_West Sacramento,city_Wilton,neighborhood_Alkali Flat,neighborhood_American River Canyon,neighborhood_American River Canyon North,neighborhood_Arcade Creek,neighborhood_Birdcage Heights,neighborhood_Boulevard Park,neighborhood_Broderrick/Bryte,neighborhood_Campus Commons,neighborhood_Cannon Industrial Park,neighborhood_Capital Village,neighborhood_Carelton Tract,neighborhood_Central Oak Park,neighborhood_Chase,neighborhood_College-Glen,neighborhood_Colonial Heights,neighborhood_Colonial Manor,neighborhood_Colonial Village,neighborhood_Cordova Lane,neighborhood_Cordova Meadows,neighborhood_Cordova Vineyards,neighborhood_Creekside,neighborhood_Curtis Park,neighborhood_Del Paso Heights,neighborhood_Del Paso Park,neighborhood_Downtown,neighborhood_East Del Paso Heights,neighborhood_East Sacramento,neighborhood_Elmhurst,neighborhood_Fairgrounds,neighborhood_Foothill Farms,neighborhood_Fruitridge Manor,neighborhood_Gateway Center,neighborhood_Gateway West,neighborhood_Glenwood Meadows,neighborhood_Golf Course Terrace,neighborhood_Greenhaven,neighborhood_Hagginwood,neighborhood_Land Park,neighborhood_Lighthouse,neighborhood_Little Pocket,neighborhood_Mangan Park,neighborhood_Mansion Flats,neighborhood_Marshall School,neighborhood_Meadowview,neighborhood_Medical Center,neighborhood_Metro Center,neighborhood_Michigan-Glide-Sutter,neighborhood_Midtown - Winn Park Capital Avenue,neighborhood_Mills Ranch,neighborhood_Natomas Creek,neighborhood_Natomas Crossing,neighborhood_Natomas Park,neighborhood_New Era Park,neighborhood_Newton Booth,neighborhood_Noralto,neighborhood_North City Farms,neighborhood_North Highlands,neighborhood_North Oak Park,neighborhood_Northeast Village,neighborhood_Northgate,neighborhood_Northwest,neighborhood_Northwest Village,neighborhood_Oak Knoll,neighborhood_Old North Sacramento,neighborhood_Old West Sacramento,neighborhood_Park Oaks,neighborhood_Parkway,neighborhood_Pocket,neighborhood_Port North,neighborhood_RP Sports Complex,neighborhood_Richmond Grove,neighborhood_Rio del Oro,neighborhood_River Gardens,neighborhood_River Park,neighborhood_Robla,neighborhood_Rusch Park,neighborhood_Sierra Oaks,neighborhood_South City Farms,neighborhood_South Hagginwood,neighborhood_South Land Park,neighborhood_South Natomas,neighborhood_South Oak Park,neighborhood_South White Rock,neighborhood_Southside Park,neighborhood_Southwest Village,neighborhood_Sun River,neighborhood_Sunridge Anatolia,neighborhood_Sunridge Park,neighborhood_Sunrise Oaks,neighborhood_Sunrise Ranch,neighborhood_Swanston Estates,neighborhood_Sylvan Old Auburn Road,neighborhood_Tahoe Park,neighborhood_Tahoe Park East,neighborhood_Tahoe Park South,neighborhood_Tallac Village,neighborhood_Triangle,neighborhood_Upper Land Park,neighborhood_Valley High-North Laguna,neighborhood_Village 11,neighborhood_Village 12,neighborhood_Village 2,neighborhood_Village 5,neighborhood_Village 7,neighborhood_Village 9,neighborhood_Vineyard,neighborhood_Walnut Wood,neighborhood_West Tahoe Park,neighborhood_Westlake,neighborhood_White Rock,neighborhood_Willowcreek,neighborhood_Wills Acres,neighborhood_Woodlake,home_type_Apartment,home_type_Multi Family,home_type_Single Family Home,home_type_Townhouse,smoking_No smoking,living_room_Living room,dishwasher_Dishwasher,microwave_Microwave,refrigerator_Refrigerator,on_site_maint_Online Maintenance Portal,on_site_mng_On-Site Management,laundry_Hookups,laundry_In Unit,laundry_Laundry Facilities,laundry_OnSiteLaundry,laundry_Shared,air_con_Air Conditioning,disposal_Disposal,dryer_Dryer,patio_Patio,pool_Pool,balcony_Balcony,washer_Washer,washer_Washer Dryer Hookup,basketball_Basketball Court,ceiling_fan_Ceiling Fan,fireplace_Fireplace,fitness_Fitness Center,playground_Playground,floor_types_Concrete,floor_types_Hardwood,floor_types_Laminate,